In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate() 
using RobotZoo:Quadrotor
using RobotDynamics
import ForwardDiff as FD
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

import MeshCat as mc
using ColorTypes
using GeometryBasics: HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

include(joinpath(@__DIR__,"utils/cartpole_animation.jl"))

# using Plots
# using Printf

  Activating environment at `~/Documents/REx_Lab/tinympc-julia/codegen_examples/Project.toml`


animate_cartpole (generic function with 1 method)

In [2]:
# Define ground truth cartpole ODE for simulation
function dynamics(params::NamedTuple, x::Vector, u)
    # cartpole physical parameters 
    mc, mp, l = params.mc, params.mp, params.l
    g = 9.81
    
    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])

    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    return [qd;qdd]

end

function rk4(params::NamedTuple, x::Vector,u,dt::Float64)
    k1 = dt*dynamics(params, x, u)
    k2 = dt*dynamics(params, x + k1/2, u)
    k3 = dt*dynamics(params, x + k2/2, u)
    k4 = dt*dynamics(params, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

rk4 (generic function with 1 method)

In [3]:
# Define problem parameters
params = (mc = 1.2, mp = 0.16, l = 0.55)

# states and control sizes 
nx = 4 
nu = 1 

# desired x and u
xgoal = [0, pi, 0, 0]
ugoal = [0]

# initial condition
x0 = xgoal + [0.1, 0.1, 0.0, 0.0]

# simulation size 
dt = 0.01
tf = 10.0 
t_vec = 0:dt:tf
Nsim = length(t_vec)
Nh = 10 # horizon length

# define cost matrices
Q = diagm([1,1,0.05,1])
Qf = 10*Q
R = [0.1]
R = reshape(R,(nu,nu))
ρ = 0.1

# Determine linearized dynamics of the cartpole about its upright position
A = FD.jacobian(dx -> rk4(params, dx, ugoal, dt), xgoal)
B = FD.jacobian(du -> rk4(params, xgoal, du, dt), ugoal)

xmin = -5 .* ones((nx,Nh))
xmax = 5 .* ones((nx,Nh))
umin = -5 .* ones((nu,Nh))
umax = 5 .* ones((nu,Nh))

# Solver options
abs_pri_tol = 1e-3
rel_pri_tol = 1e-3
max_iter = 100
verbose = 1

tinympc_dir = "/Users/anoushkaalavill/Documents/REx_Lab/TinyMPC/"
output_dir = "/generated_code"

"/generated_code"

In [4]:
# Create wrappers for the C backend
using Libdl
# create pointer to shared library
lib = dlopen("/Users/anoushkaalavill/Documents/REx_Lab/TinyMPC/generated_code/build/src/libtiny_wrapper.dylib")

# create pointers to tiny_wrapper functions
set_x = dlsym(lib, "set_x")
function ccall_set_x!(x0)
    @ccall $set_x(x0::Ptr{Cfloat})::Cvoid 
end

set_xref = dlsym(lib, "set_xref")
function ccall_set_xref!(x0)
    @ccall $set_xref(xref::Ptr{Cfloat})::Cvoid 
end

reset_dual_variables = dlsym(lib, "reset_dual_variables")
function ccall_reset_dual_variables!()
    @ccall $reset_dual_variables()::Cvoid
end

call_tiny_solve = dlsym(lib, "call_tiny_solve")
function ccall_tiny_solve!()
    @ccall $call_tiny_solve()::Cvoid
end

get_x = dlsym(lib, "get_x")
function ccall_get_x!(x_soln)
    @ccall $get_x(x_soln::Ptr{Cfloat})::Cvoid
end

get_u = dlsym(lib, "get_u")
function ccall_get_u!(u_soln)
    @ccall $get_u(u_soln::Ptr{Cfloat})::Cvoid
end

edit_x = dlsym(lib, "edit_x")
function ccall_edit_x!(x)
    @ccall $edit_x(x::Ptr{Cfloat})::Cvoid
end

ccall_edit_x! (generic function with 1 method)

In [7]:
# # tests
# @show x0
# ccall_set_x!(x0)
# ccall_reset_dual_variables!()
# ccall_tiny_solve!()

xref=1*xgoal
for i=1:Nh-1
    xref = hcat(xref, xgoal)
end
@show xref
xref = Array{Float32}(xref)
ccall_set_xref!(xref)
# x_soln = Array{Float32}(x_soln)
# ccall_print_x!(x_soln)
# @show Array{Float32}(x_soln)

xref = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 3.141592653589793 3.141592653589793 3.141592653589793 3.141592653589793 3.141592653589793 3.141592653589793 3.141592653589793 3.141592653589793 3.141592653589793 3.141592653589793; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref resu

In [8]:
# MPC loop

x_hist = [zeros(nx) for i = 1:Nsim] # store x values for sim

for k=1:Nsim
    x_hist[k] = x0
    # set initial condition and enforce float type
    x0 = Array{Float32}(x0)
    ccall_set_x!(x0)

    ccall_set_xref!(xref)

    # reset dual variables
    ccall_reset_dual_variables!()

    # solve
    ccall_tiny_solve!()

    # get u solution
    u_soln = zeros((nu,Nh-1))
    u_soln = Array{Float32}(u_soln)
    ccall_get_u!(u_soln)
    # @show u_soln[1,1]

    x0 = rk4(params,x0,u_soln[1,1],dt)
    @show x0
end

set_x result:  0.100000
set_x result:  3.241593
set_x result:  0.000000
set_x result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref resu

[0.10061439001139387, 3.248489254742749, -0.006723060704028698, 0.1341521510790502]
u_soln:  -0.705143
u_soln:  -0.705735
u_soln:  -0.705483
u_soln:  -0.704414
u_soln:  -0.702645
u_soln:  -0.700233
u_soln:  -0.697234
u_soln:  -0.693743
u_soln:  -0.689832
set_x result:  0.100614
set_x result:  3.248489
set_x result:  -0.006723
set_x result:  0.134152
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000



u_soln:  -0.773681
u_soln:  -0.771017
u_soln:  -0.767592
u_soln:  -0.763463
u_soln:  -0.758726
u_soln:  -0.753443
u_soln:  -0.747662
u_soln:  -0.741491
u_soln:  -0.734992
set_x result:  0.099006
set_x result:  3.260642
set_x result:  -0.039726
set_x result:  0.214696
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
se

[0.09496066993626305, 3.27881888847553, -0.07662950877169261, 0.3070260592519243]
u_soln:  -0.880440
u_soln:  -0.872629
u_soln:  -0.864216
u_soln:  -0.855279
u_soln:  -0.845870
u_soln:  -0.836093
u_soln:  -0.825990
u_soln:  -0.815619
u_soln:  -0.805051
set_x result:  0.094961
set_x result:  3.278819
set_x result:  -0.076630
set_x result:  0.307026
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
se


u_soln:  -1.031469
u_soln:  -1.016335
u_soln:  -1.000848
u_soln:  -0.985058
u_soln:  -0.969047
u_soln:  -0.952867
u_soln:  -0.936574
u_soln:  -0.920245
u_soln:  -0.903943
set_x result:  0.088141
set_x result:  3.304006
set_x result:  -0.119315
set_x result:  0.415855
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
se

...

u_soln:  -2.533436
u_soln:  -1.856998
u_soln:  -1.086561
u_soln:  -0.223652
u_soln:  0.730389
u_soln:  1.774332
u_soln:  2.907112
u_soln:  4.127733
u_soln:  5.272219
set_x result:  -7.313615
set_x result:  24.394873
set_x result:  -3.796536
set_x result:  11.908052
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_x


u_soln:  -3.157639
u_soln:  -2.368900
u_soln:  -1.484791
u_soln:  -0.506767
u_soln:  0.563969
u_soln:  1.726282
u_soln:  2.979294
u_soln:  4.322206
u_soln:  5.374384
set_x result:  -7.598596
set_x result:  25.277342
set_x result:  -4.240623
set_x result:  12.992350
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_

[-7.724803029663851, 25.66322547262032, -4.136455628509373, 12.642509186222501]
u_soln:  0.416126
u_soln:  1.004238
u_soln:  1.700354
u_soln:  2.502846
u_soln:  3.500652
u_soln:  3.934864
u_soln:  4.279112
u_soln:  4.623453
u_soln:  4.967947
set_x result:  -7.724803
set_x result:  25.663225
set_x result:  -4.136456
set_x result:  12.642509
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref r

Excessive output truncated after 524468 bytes.


u_soln:  1.910038
u_soln:  2.268028
u_soln:  2.625443
u_soln:  2.982349
u_soln:  3.338909
u_soln:  3.695248
u_soln:  4.051491
u_soln:  4.407822
u_soln:  4.764320
set_x result:  -7.992081
set_x result:  26.490114
set_x result:  -3.484418
set_x result:  10.995296
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  3.141593
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref result:  0.000000
set_xref

In [9]:
# Simulate
display(animate_cartpole(x_hist, dt))

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("pole" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "b" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x62], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "cart" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x63, 0x61, 0x72, 0x74], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "a" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x61], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), Set{Any}(), ip"127.0.0.1", 8700))